In [1]:
# Tanqiu Jiang
# DSCC 440 Data Mining Implementation Project

# Import the UCI adult dataset
import pandas as pd
import numpy as np
import time
df = pd.read_csv (r'adult.csv')
print (df)

       age     workclass  fnlwgt     education  educational-num  \
0       25       Private  226802          11th                7   
1       38       Private   89814       HS-grad                9   
2       28     Local-gov  336951    Assoc-acdm               12   
3       44       Private  160323  Some-college               10   
4       18             ?  103497  Some-college               10   
...    ...           ...     ...           ...              ...   
48837   27       Private  257302    Assoc-acdm               12   
48838   40       Private  154374       HS-grad                9   
48839   58       Private  151910       HS-grad                9   
48840   22       Private  201490       HS-grad                9   
48841   52  Self-emp-inc  287927       HS-grad                9   

           marital-status         occupation relationship   race  gender  \
0           Never-married  Machine-op-inspct    Own-child  Black    Male   
1      Married-civ-spouse    Farming-fishin

In [2]:
def FPGrowth (df,min_support,result):
    tic = time.perf_counter()
    i=0
    C1 = {}
    L1 = []
    tree = []
    if (min_support < 1): 
        cnt = df[df.columns.values[0]].count()
        cnt = int(cnt * min_support)
        min_support = cnt
        print("The min_support you set is:",min_support)
    newdf = df.copy()
    replaceItems(newdf)
    for elements in df.columns.values:
        header = df.columns.values[i]
        FirstRound(df,header,C1)
        i=i+1
    print("The F-list is:",FindFrequent(C1,min_support,L1))
    i=0
    j=0
    k=0
    m=0
    value = 0
    myFtable = HashTable()
    mytable = HashTable()
    ItemSet = L1
    C = {}
    for j in range (len(newdf[newdf.columns.values[k]].values)-1):
        i = 0
        m += 1
        temp = ""
        if m % 1000 == 1:
            print("working on line:", j)
        for elements in ItemSet:
            #count = 3
            value = 0
            for char in ItemSet[i]:
                value += ord(char)
                #print(count)
                for k in range (len(newdf.iloc[j])):
                    if (newdf.iloc[j][k] == value):
                        temp += ItemSet[i] + " "
                        mytable[temp] = temp
                        myFtable[temp] += 1
            i = i + 1
        j = j + 1
    for a in range(100000):
        if myFtable.arr[a] > 0:
            #print(myFtable.arr[a])
            #print(str(mytable.arr[a]))
            C[str(mytable.arr[a])] = myFtable.arr[a]
    #print(C)
    j = 0
    Items = {}
    for j in range (len(L1)-1):
        j = j+1
        FindItemSet(L1,j,Items,C)
    #print(Items)
    FinalAnwser = L1
    FindFrequent(Items,min_support,FinalAnwser)
    result += FinalAnwser
    print("The frequent items are:",FinalAnwser,"with min_support of",min_support)
    toc = time.perf_counter()
    print("My FP-Growth algorithm takes",int((toc - tic)/3600),"Hours",int((toc - tic)/60%60),"minutes",int(toc - tic)%60,"seconds to run! (Min_Support =",min_support,")")
    return (result)

In [3]:
def FindItemSet(L1,Freq,FrequentItem,C):  
    tree = []
    GenerateTree(L1,tree,Freq)
    #print(tree)
    x=0
    treeTable = HashTable()
    treeFTable = HashTable()
    #FrequentItem = {}
    for x in range (len(tree)):
        temp = ""
        myString = ""
        for char in tree[x]:
            if char == " ":
                #print(temp)
                addSpace = True
                for y in range (len(L1)-Freq):
                    #print (L1[y])
                    if temp == L1[y]:
                        #print(temp)
                        myString += temp + " "
                        #print(myString)
                        treeTable[myString+L1[len(L1)-Freq]] = myString+L1[len(L1)-Freq]
                        if (C.get(tree[x])!=None):
                            #print(tree[x])
                            treeFTable[myString+L1[len(L1)-Freq]] += C.get(tree[x])
                            #print(myString+L1[len(L1)-1])
                        if (C.get(tree[x])==None):
                            treeFTable[myString+L1[len(L1)-Freq]] = 0
                        temp = ""
                        addSpace = False
                if addSpace:
                    temp += char
            else:
                temp += char
    for a in range(100000):
        if treeFTable.arr[a] > 0:
            #print(myFtable.arr[a])
            #print(str(mytable.arr[a]))
            FrequentItem[str(treeTable.arr[a])] = treeFTable.arr[a]
    return(FrequentItem)

In [4]:
def replaceItems(dataframe):
    i=0
    for i in range (len(df.columns.values)): 
        y = []
        for ind,row in dataframe.iterrows():
            x = 0
            for char in str(row[i]):
                x += ord(char)
            for char in (df.columns.values[i]):
                x += ord(char)
            for char in (": "):
                x += ord(char)
            y.append(x)
        dataframe[df.columns.values[i]] = y
    return(dataframe)

In [5]:
class HashTable:  
    def __init__(self):
        self.MAX = 100000
        self.arr = [0 for i in range(self.MAX)]
        
    def get_hash(self, key):
        hash = 0
        y = 3
        if isinstance(key, (int, np.int64)):
            return key % self.MAX
        else:
            for char in key:
                hash += (ord(char)+77)*y*89
                y = y+3
            return hash % self.MAX
    
    def __getitem__(self, index):
        h = self.get_hash(index)
        return self.arr[h]
    
    def __setitem__(self, key, val):
        h = self.get_hash(key)
        self.arr[h] = val

In [6]:
def FirstRound(df,key,C1):
    i=0
    table = HashTable()
    for element in df[key]:
        table[df[key][i]] = df[key][i]
        i=i+1
    Ftable = HashTable()
    i=0
    for element in df[key]:
        Ftable[df[key][i]] += 1
        i=i+1
    for i in range(100000):
        if Ftable.arr[i] > 0:
            C1[key+": "+str(table.arr[i])] = Ftable.arr[i]
    return(C1)

In [7]:
def FindFrequent(itemset,int,L1):
    sortedDict = sorted(itemset.items(), key=lambda x: x[1], reverse=True)
    L1Dict = {}
    for i in sortedDict:
        if i[1]>int:
            L1Dict[i[0]] = i[1]
            L1.append(i[0])
    return(L1)

In [8]:
def GenerateTree(L1,tree,number):
    from itertools import combinations
    a =0
    for a in range (len(L1)):
        a=a+1
        comb = combinations(L1, a) 
        for i in list(comb):
            temp = ""
            #print(a)
            if i[a-1] == L1[len(L1)-number]:
                for x in range (a):
                    temp += i[x] + " "
            if temp != "":
                #print(temp)
                tree.append(temp)
    return(tree)

In [10]:
FinalResult = []
FPGrowth(df,25000,FinalResult)
print("As we can see the final result is:",FinalResult)

The F-list is: ['capital-loss: 0', 'capital-gain: 0', 'native-country: United-States', 'race: White', 'income: <=50K', 'workclass: Private', 'gender: Male']
working on line: 0
working on line: 1000
working on line: 2000
working on line: 3000
working on line: 4000
working on line: 5000
working on line: 6000
working on line: 7000
working on line: 8000
working on line: 9000
working on line: 10000
working on line: 11000
working on line: 12000
working on line: 13000
working on line: 14000
working on line: 15000
working on line: 16000
working on line: 17000
working on line: 18000
working on line: 19000
working on line: 20000
working on line: 21000
working on line: 22000
working on line: 23000
working on line: 24000
working on line: 25000
working on line: 26000
working on line: 27000
working on line: 28000
working on line: 29000
working on line: 30000
working on line: 31000
working on line: 32000
working on line: 33000
working on line: 34000
working on line: 35000
working on line: 36000
worki

In [ ]:
# Please Disregard anything below this point
# Those are pieces that were once used for testing purposes

In [94]:
    i=0
    j=0
    k=0
    m=0
    value = 0
    myFtable = HashTable()
    mytable = HashTable()
    ItemSet = L1
    C = {}
    for j in range (len(newdf[newdf.columns.values[k]].values)-1):
        i = 0
        m += 1
        temp = ""
        if m % 1000 == 1:
            print("working on line:", j)
        for elements in ItemSet:
            #count = 3
            value = 0
            for char in ItemSet[i]:
                value += ord(char)
                #print(count)
                for k in range (len(newdf.iloc[j])):
                    if (newdf.iloc[j][k] == value):
                        temp += ItemSet[i] + " "
                        mytable[temp] = temp
                        myFtable[temp] += 1
            i = i + 1
        j = j + 1
    for a in range(100000):
        if myFtable.arr[a] > 0:
            #print(myFtable.arr[a])
            #print(str(mytable.arr[a]))
            C[str(mytable.arr[a])] = myFtable.arr[a]
    print(C)

working on line: 0
working on line: 1000
working on line: 2000
working on line: 3000
working on line: 4000
working on line: 5000
working on line: 6000
working on line: 7000
working on line: 8000
working on line: 9000
working on line: 10000
working on line: 11000
working on line: 12000
working on line: 13000
working on line: 14000
working on line: 15000
working on line: 16000
working on line: 17000
working on line: 18000
working on line: 19000
working on line: 20000
working on line: 21000
working on line: 22000
working on line: 23000
working on line: 24000
working on line: 25000
working on line: 26000
working on line: 27000
working on line: 28000
working on line: 29000
working on line: 30000
working on line: 31000
working on line: 32000
working on line: 33000
working on line: 34000
working on line: 35000
working on line: 36000
working on line: 37000
working on line: 38000
working on line: 39000
working on line: 40000
working on line: 41000
working on line: 42000
working on line: 43000
w

In [215]:
j = 0
Items = {}
for j in range (len(L1)-1):
    j = j+1
    FindItemSet(L1,j,Items)
#print(Items)
FinalAnwser = L1
FindFrequent(Items,35000,FinalAnwser)
print("The frequent items are:",FinalAnwser,"with min_support of 35000")

The frequent items are: ['capital-loss: 0', 'capital-gain: 0', 'native-country: United-States', 'race: White', 'income: <=50K', 'capital-loss: 0 capital-gain: 0', 'capital-loss: 0 native-country: United-States', 'capital-loss: 0 race: White', 'capital-loss: 0 capital-gain: 0 native-country: United-States', 'capital-loss: 0 capital-gain: 0 race: White', 'capital-loss: 0 income: <=50K', 'capital-loss: 0 capital-gain: 0', 'capital-loss: 0 native-country: United-States', 'capital-loss: 0 race: White', 'capital-loss: 0 capital-gain: 0 native-country: United-States', 'capital-loss: 0 capital-gain: 0 race: White', 'capital-loss: 0 income: <=50K'] with min_support of 35000


In [192]:
C["capital-loss: 0 native-country: United-States "]

3689

In [200]:
tree = []
GenerateTree(L1,tree,4)

['capital-gain: 0 ', 'capital-loss: 0 capital-gain: 0 ']

In [205]:
tree = []
GenerateTree(L1,tree,4)
#print(tree)
x=0
treeTable = HashTable()
treeFTable = HashTable()
FrequentItem = {}
for x in range (len(tree)):
    temp = ""
    myString = ""
    for char in tree[x]:
        if char == " ":
            #print(temp)
            addSpace = True
            for y in range (len(L1)-4):
                #print (L1[y])
                if temp == L1[y]:
                    #print(temp)
                    myString += temp + " "
                    #print(myString)
                    treeTable[myString+L1[len(L1)-4]] = myString+L1[len(L1)-4]
                    if (C.get(tree[x])!=None):
                        #print(tree[x])
                        treeFTable[myString+L1[len(L1)-4]] += C.get(tree[x])
                        #print(myString+L1[len(L1)-1])
                    if (C.get(tree[x])==None):
                        treeFTable[myString+L1[len(L1)-4]] = 0
                    temp = ""
                    addSpace = False
            if addSpace:
                temp += char
        else:
            temp += char
for a in range(100000):
    if treeFTable.arr[a] > 0:
        #print(myFtable.arr[a])
        #print(str(mytable.arr[a]))
        FrequentItem[str(treeTable.arr[a])] = treeFTable.arr[a]
print(FrequentItem)

{'capital-loss: 0 capital-gain: 0': 42483}


In [ ]:
i = 0
Grow = HashTable()
GrowF = HashTable()
temp = ""
x = 0
for i in range (len(L1)):
    for x in range (i+1):
        print(i,x)
        temp =""
        temp += L1[i-x]+" "
        print (temp)
        x+=1
    i+=1
    #temp += ""
    #Grow[L1[i]] = L1[i]
    
    i=i+1

In [93]:
i=0
C1 = {}
L1 = []
tree = []
newdf = df.copy()
replaceItems(newdf)
for elements in df.columns.values:
    header = df.columns.values[i]
    FirstRound(df,header,C1)
    i=i+1
print("The F-list is:",FindFrequent(C1,35000,L1))
#print("The tree looks like: ")
#GenerateTree(L1,tree)
#print(tree)

The F-list is: ['capital-loss: 0', 'capital-gain: 0', 'native-country: United-States', 'race: White', 'income: <=50K']
